In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
import torch

In [8]:
#Class Value pour nous permettre de conserver une trace des parents, enfants etc - permettra d'appliquer une
#backpropagation
class Value:
    
    def __init__(self, data, _children = (), _op = '', label = ''):
        self.data = data
        self.grad = 0.0 #0 au début car on part du principe qu'aucune valeur ne change l'output
        self._backward = lambda: None #par défaut, fonction vide qui ne fait rien
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __repr__(self): #permet de print quelque chose de visible, renverrait autrement un objet value et son
        return f"Value(data={self.data})" #adresse dans la RAM
    
    def __radd__(self, other):
        return self + other
    
    def __add__(self, other): # python appelera a.__add__(b)
        other = other if isinstance(other, Value) else Value(other) #si other != objet Value, on le traduit en Val
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        
        out._backward = _backward #on veut renvoyer la fonction, pas ce qu'elle renvoie
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Supporte que des ints et des floats pour l'instant"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other -1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
                        #(x, ) => tuple avec un seul objet
        
        def _backward():
            self.grad += out.data * out.grad
        
        out._backward = _backward
            
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self, ), 'tanh')
                        #(x, ) => tuple avec un seul objet
        
        def _backward():
            self.grad += (1 - t ** 2) * out.grad
        
        out._backward = _backward
            
        return out
    
    def backward(self):

        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        
        for node in reversed(topo):
            node._backward()

In [10]:
#input x1, x2
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

#poids w1, w2
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

#biais du neuronne - pris dans le livre neural network from scratch
b = Value(6.8813735870195432, label = 'b')
 
#x1 * w1 + x2 * w2 + b à petit pas pour avoir des pointeurs au fur et à mesure - permet la backpropagation
x1w1 = x1 * w1; x1w1.label = 'x1*w1'
x2w2 = x2 * w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh() ; o.label = 'o'

o.backward()

In [128]:
o.grad = 1.0 # de base _grad vaut 0.0 dans la class, set le premier grad

topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(o)

for node in reversed(topo):
    node._backward()

In [ ]:
#input x1, x2
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

#poids w1, w2
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

#biais du neuronne, pareillement
b = Value(6.8813735870195432, label = 'b')
 
#x1 * w1 + x2 * w2 + b à petit pas pour avoir des pointeurs au fur et à mesure, même raison
x1w1 = x1 * w1; x1w1.label = 'x1*w1'
x2w2 = x2 * w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'

# --------
e = (2*n).exp()
o = (e - 1) / (e + 1)
# --------

o.backward()

In [13]:
#Comparaison de nos résultats avec Pytorch :

x1 = torch.Tensor([2.0]).double()                    ; x1.requires_grad = True #.double permet un dtype float64
x2 = torch.Tensor([0.0]).double()                    ; x2.requires_grad = True # ce qui permet une plus grande
w1 = torch.Tensor([-3.0]).double()                   ; w1.requires_grad = True # précision dans les calculs
w2 = torch.Tensor([1.0]).double()                    ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()      ; b.requires_grad = True
n = x1 * w1 + x2 * w2 + b
o = torch.tanh(n)

print(o.data.item()) #.item() retire l'élément du tensor dans lequel il se trouve
o.backward()

print('------')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
------
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [16]:
o

tensor([0.7071], dtype=torch.float64, grad_fn=<TanhBackward0>)

In [17]:
o.item() #résultats similaires !

0.7071066904050358

In [18]:
#réseau de neuronne
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
        
    def __call__(self, x):
        # w * x + b
        zip(self.w, x) #zip prends deux liste sur lequel itérer et réunis la valeur de 
        #chaque  index dans un tuple sur le quel il itère
        
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs #permet de renvoyer l'élément de la liste si la liste n'a qu'un
        # éléments
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [19]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=-0.21693394468520402)

In [20]:
n.parameters()

[Value(data=0.6258066781944112),
 Value(data=0.7987031621564893),
 Value(data=0.09874373474922704),
 Value(data=0.655762367936479),
 Value(data=0.36230682594086905),
 Value(data=0.9157348679634445),
 Value(data=0.8551018539949453),
 Value(data=-0.262877233170276),
 Value(data=0.883554291072516),
 Value(data=0.3007726154553507),
 Value(data=-0.284868299948009),
 Value(data=-0.4165757451600183),
 Value(data=-0.6560178251309474),
 Value(data=-0.024610934113758587),
 Value(data=0.5966987031545588),
 Value(data=0.2430501218985912),
 Value(data=0.3281192076786523),
 Value(data=0.881698025942619),
 Value(data=0.5172654837050137),
 Value(data=-0.9273639337910244),
 Value(data=0.13739527939731744),
 Value(data=0.26674187199291177),
 Value(data=0.8906302090038782),
 Value(data=-0.010333371961902538),
 Value(data=0.21814426959594901),
 Value(data=-0.017910807235872683),
 Value(data=-0.33797433115581477),
 Value(data=-0.3981355767720829),
 Value(data=0.044809735804365536),
 Value(data=-0.160127533

In [21]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0] # target que l'on souhaite atteindre

In [144]:
for k in range(20):
    
    #forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    #backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    #update 
    for p in n.parameters():
        p.data += 0.05 * p.grad
    
    print(k, loss.data)

0 7.906105206602827
1 7.911093636568248
2 7.915591602995168
3 7.919667452022988
4 7.923377443073448
5 7.926768302647951
6 7.929879157410131
7 7.932743015665671
8 7.935387915665355
9 7.937837824948708
10 7.940113351467943
11 7.942232310870031
12 7.944210182749806
13 7.946060480405518
14 7.947795052625903
15 7.949424331638477
16 7.9509575380903375
17 7.9524028514957745
18 7.953767552745635
19 7.955058143873199


In [145]:
ypred # après computation, très proche des targets : réseau fonctionnel !

[Value(data=-0.994551995802675),
 Value(data=-0.9930340688316286),
 Value(data=-0.9955061745907926),
 Value(data=-0.9941794706718133)]